<a href="https://colab.research.google.com/github/physteacherlab/qe_data_science/blob/Physics-Data-Analysis/QE_DS_classwork_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задачи для самостоятельной работы по анализу данных

В этом разделе 5 задач, которые помогут закрепить навыки загрузки, обработки, анализа и визуализации данных на Python.

Для каждой задачи предусмотрен свой CSV-файл с исходными данными.
Внимательно читайте условия и используйте подсказки к решению.

## Задача 1: Очистка временного ряда с пропусками и выбросами

Дан csv-файл с двумя столбцами: "time" (временные точки) и "value" (значения ряда). Есть пропуски и выбросы.


### Задание:
- Загрузите данные с помощью pandas.
- Обработайте пропуски методом интерполяции по времени.
- Примените медианный фильтр (с окном 3 или 5) для замены выбросов.
- Постройте график до и после очистки.

### Подсказки:
- Используйте `pandas.read_csv`, `df.interpolate(method='time')`.
- Для медианного фильтра — `scipy.signal.medfilt`.
- Для визуализации — `matplotlib.pyplot.plot`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1) Загрузка данных
df = pd.read_csv('task1_time_series.csv', parse_dates=['time'])

# Установим время как индекс (удобно для интерполяции по времени)
df = df.set_index('time')

print(df.head())
print(df.isna().sum())  # смотрим пропуски

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(df.index, df['value'], label='Исходные данные')
plt.xlabel('time')
plt.ylabel('value')
plt.title('Временной ряд: исходные данные')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Интерполяция по времени
df_interp = df.copy()
df_interp['value'] = df_interp['value'].interpolate(method='time')

plt.figure(figsize=(10, 4))
plt.plot(df.index, df['value'], 'o', label='До интерполяции', alpha=0.5)
plt.plot(df_interp.index, df_interp['value'], '-', label='После интерполяции')
plt.xlabel('time')
plt.ylabel('value')
plt.title('Интерполяция пропусков по времени')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from scipy.signal import medfilt

# Применим медианный фильтр (окно 5 точек)
df_filt = df_interp.copy()
df_filt['value_filt'] = medfilt(df_filt['value'], kernel_size=5)

plt.figure(figsize=(10, 4))
plt.plot(df_interp.index, df_interp['value'], label='После интерполяции', alpha=0.6)
plt.plot(df_filt.index, df_filt['value_filt'], label='После медианного фильтра', linewidth=2)
plt.xlabel('time')
plt.ylabel('value')
plt.title('Сглаживание выбросов медианным фильтром')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(df.index, df['value'], 'o-', label='Исходные данные', alpha=0.5)
plt.plot(df_filt.index, df_filt['value_filt'], '-', label='Очистка: интерполяция + медианный фильтр', linewidth=2)
plt.xlabel('time')
plt.ylabel('value')
plt.title('Ряд до и после очистки')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Задача 2: Сглаживание и НЧ-фильтрация временного сигнала

Дан csv с временным рядом: "time", "signal" — сигнал содержит полезный тренд, гармонический компонент и шум.

### Задание:
- Загрузите данные из csv.
- Выполните скользящее среднее и экспоненциальное сглаживание.
- Посчитайте спектр через БПФ, выделите доминирующую частоту.
- Реализуйте низкочастотный фильтр: обнулите спектр выше выбранного порога и восстановите сигнал.
- Постройте графики: исходный сигнал, оба вида сглаженных сигналов, спектр, отфильтрованный сигнал.

## Решение задачи 2

1) Выполним загрузку данных и первичный просмотр с помощью `pandas`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Загрузка данных
df = pd.read_csv('task2_signal.csv', parse_dates=['time'])
df = df.set_index('time')

df.head(), df.describe()

2) Выполним визуализацию исходного сигнала с помощью `matplotlib`

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(df.index, df['signal'], label='Исходный сигнал')
plt.xlabel('time')
plt.ylabel('signal')
plt.title('Исходный временной сигнал')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

3) Выполним сглашивание по методу "скользящее среднее"


In [ ]:
window = 5  # размер окна в точках

df['rolling_mean'] = df['signal'].rolling(window=window, center=True).mean()

plt.figure(figsize=(10, 4))
plt.plot(df.index, df['signal'], label='Исходный сигнал', alpha=0.5)
plt.plot(df.index, df['rolling_mean'], label=f'Скользящее среднее (окно={window})', linewidth=2)
plt.xlabel('time')
plt.ylabel('signal')
plt.title('Скользящее среднее')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

4) Выполним экспоненциальное сглаживание


In [ ]:
alpha = 0.2  # сглаживающий параметр

df['exp_smooth'] = df['signal'].ewm(alpha=alpha, adjust=False).mean()

plt.figure(figsize=(10, 4))
plt.plot(df.index, df['signal'], label='Исходный сигнал', alpha=0.5)
plt.plot(df.index, df['exp_smooth'], label=f'Экспоненциальное сглаживание (alpha={alpha})', linewidth=2)
plt.xlabel('time')
plt.ylabel('signal')
plt.title('Экспоненциальное сглаживание')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

5) Сделаем расчет спектра (БПФ) и поиск доминирующей частоты.
Предположим равномерную дискретизацию по времени. Для частот используем `np.fft.rfftfreq`.

In [ ]:
signal = df['signal'].values
N = len(signal)

# шаг по времени (в секундах или условных единицах)
dt = (df.index[1] - df.index[0]).total_seconds()  # для минут можно оставить так или использовать минуты

# Прямое БПФ
fft_vals = np.fft.rfft(signal)
freqs = np.fft.rfftfreq(N, d=dt)

# Амплитудный спектр
amplitude = np.abs(fft_vals)

plt.figure(figsize=(10, 4))
plt.plot(freqs, amplitude)
plt.xlabel('Частота')
plt.ylabel('Амплитуда')
plt.title('Амплитудный спектр сигнала')
plt.grid(True)
plt.tight_layout()
plt.show()

# Поиск доминирующей частоты (кроме нуля)
dominant_idx = np.argmax(amplitude[1:]) + 1
dominant_freq = freqs[dominant_idx]
print("Доминирующая частота:", dominant_freq)

6) Построим простой низкочастотный фильтр в частотной области. Сделаем «идеальный» НЧ-фильтр: обрежем все частоты выше порога `f_cut`.

In [ ]:
# Задаем частоту среза (например, чуть выше найденной доминирующей)
f_cut = dominant_freq * 1.5

fft_filtered = fft_vals.copy()
fft_filtered[freqs > f_cut] = 0  # зануляем высокие частоты

# Обратное БПФ
signal_filtered = np.fft.irfft(fft_filtered, n=N)

df['signal_filtered'] = signal_filtered

plt.figure(figsize=(10, 4))
plt.plot(df.index, df['signal'], label='Исходный сигнал', alpha=0.4)
plt.plot(df.index, df['signal_filtered'], label='После НЧ-фильтра в БПФ', linewidth=2)
plt.xlabel('time')
plt.ylabel('signal')
plt.title('Сигнал до и после частотной фильтрации')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

7) Сравним все примененные методы сглаживания.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df.index, df['signal'], label='Исходный сигнал', alpha=0.3)
plt.plot(df.index, df['rolling_mean'], label='Скользящее среднее', linewidth=2)
plt.plot(df.index, df['exp_smooth'], label='Экспоненциальное сглаживание', linewidth=2)
plt.plot(df.index, df['signal_filtered'], label='НЧ-фильтр (БПФ)', linewidth=2)
plt.xlabel('time')
plt.ylabel('signal')
plt.title('Сравнение методов сглаживания и фильтрации')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Задача 3: Подбор параметров модели и доверительные интервалы

В csv два столбца: "x", "y" — экспериментальные данные, \(y=f(x) + \) шум.

### Задание:
- Загрузите данные.
- Подберите параметры линейной модели двумя способами: формулы МНК и `scipy.optimize.curve_fit`.
- Рассчитайте ковариационную матрицу параметров, стандартные ошибки и 95% доверительные интервалы.
- Постройте график с данными, линией модели и доверительной полосой.

## Задача 4: Сравнение моделей и анализ остатков

Даны измерения (x, y), подходит несколько моделей: линейная, полиномиальная (степени 2 или 3), экспоненциальная.

### Задание:
- Загрузите данные.
- Постройте модели — все три типа.
- Вычислите метрики качества: RSS, R^2, AIC, BIC для каждой.
- Постройте графики остатков vs x, гистограммы остатков.
- Проверьте нормальность остатков тестом Шапиро-Уилка.
- Выберите лучшую модель, объясните выбор.

## Задача 5: Анализ данных физического эксперимента - изучение зависимости периода маятника от длины

В csv данные — измеренная длина "L" маятника и период колебаний "T", с шумом и выбросами.
### Задание:
- Загрузите данные.
- Очистите данные от выбросов с помощью метода интерквартильного размаха (IQR).
- Линеаризуйте модель: \(T^2 = \frac{4 \pi^2}{g} L\).
- Подберите параметры через линейную регрессию, оцените ускорение свободного падения и доверительный интервал.
- Постройте графики: исходные данные T(L), линеаризованный график T^2(L) с линией аппроксимации и теоретической моделью.
- Кратко прокомментируйте влияние очистки данных на оценку g.